# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [8]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [9]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("The total numberlen(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("The number of rows in the new file "+
          sum(1 for line in f))
    

# Part II. Creating an Apache Cassandra database capable of performing the required queries

## The new CSV file titled <font color=red>event_datafile_new.csv</font> is located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setting up Apache Cassandra by creating a cluster and a keyspace for Sparkify

#### Creating a Cluster locally

In [2]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace named Sparkify

In [4]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace as Sparkify

In [5]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Creating queries to answer the following three questions

#### 1. Provide the artist, song title and song's length in the music app history during sessionId = 338, and itemInSession  = 4


#### 2. Provide the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, and sessionid = 182
    

#### 3. Who are the users (provide first and last name) in the music app history that listened to the song 'All Hands Against His Own'




In [6]:
## Query 1:  Provide the artist, song title and song's length in the music app history during sessionId = 338, and itemInSession  = 4
query = "CREATE TABLE IF NOT EXISTS song_play_session "
query = query + "(session_id int,\
                  item_in_session int,\
                  artist_name text,\
                  first_name text,\
                  gender text,\
                  last_name text,\
                  song_length float,\
                  level text,\
                  location text,\
                  song_title text,\
                  user_id int,\
                  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [9]:
# Defining the insert statement for the first query
query = "INSERT INTO song_play_session (session_id, item_in_session,\
                                                artist_name, first_name,\
                                                gender, last_name,\
                                                song_length, level, location,\
                                                song_title, user_id)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Reading the csv file and inserting the data into the song_play_session table 
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], 
                                line[2], line[4], float(line[5]), line[6],
                                line[7], line[9], int(line[10])))

#### Testing the database to verify that the data have been inserted into each table and the data modeling step is correct

Remember that query 1 is: provide the artist, song title and song's length in the music app history during sessionId = 338, and itemInSession  = 4

In [10]:
## Defining the query with the appropriate select statement
query = "SELECT artist_name, song_title, song_length FROM song_play_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


Let's try answering the second question using the same table, this should not work unless we allow filtering.

The second question is: provide the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, and sessionid = 182

In [11]:
query = "SELECT artist_name, song_title, song_length FROM song_play_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


As expected, we need to create a new table that is optimizes to answer this specific question: one query = one table. 

### Let's create a new table optimized to answer the second question 

In [13]:
## Query 2: Provide the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, and sessionid = 182
query = "CREATE TABLE IF NOT EXISTS song_play_user "
query = query + "(user_id int,\
                  session_id int,\
                  item_in_session int,\
                  artist_name text,\
                  first_name text,\
                  gender text,\
                  last_name text,\
                  song_length float,\
                  level text,\
                  location text,\
                  song_title text,\
                  PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [57]:
# Defining the insert statement for the query of the second question 
query = "INSERT INTO song_play_user (user_id, session_id, item_in_session,\
                                             artist_name, first_name,\
                                             gender, last_name,\
                                             song_length, level,\
                                             location, song_title)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"


# Reading the csv file and inserting the data into the song_play_user table 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),  
                                line[0], line[1], line[2], line[4],
                                float(line[5]), line[6], line[7], line[9]))

In [58]:
## Defining the query with the appropriate select statement for the second question
query = "SELECT artist_name, song_title, item_in_session, first_name, last_name \
         FROM song_play_user \
         WHERE user_id = 10 AND session_id = 182 \
         ORDER BY session_id, item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.item_in_session, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


### Let's create a new table optimized to answer the third question 

Reminder: the third question is: Who are the users (provide first and last name) in the music app history that listened to the song 'All Hands Against His Own'

In [15]:
## Query 3: Who are the users (provide first and last name) in the music app history that listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_play_song "
query = query + "(song_title text,\
                  session_id int,\
                  item_in_session int,\
                  user_id int,\
                  artist_name text,\
                  first_name text,\
                  gender text,\
                  last_name text,\
                  song_length float,\
                  level text,\
                  location text,\
                  PRIMARY KEY (song_title, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)          

                    

In [17]:
# Defining the insert statement for the query of the third question 
query = "INSERT INTO song_play_song (song_title, session_id, item_in_session,\
                                             user_id, artist_name, first_name,\
                                             gender, last_name,\
                                             song_length, level,\
                                             location)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Reading the csv file and inserting the data into the song_play_song table 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[8]), int(line[3]),  
                                int(line[10]), line[0], line[1], line[2],
                                line[4], float(line[5]), line[6], line[7]))

In [19]:
## Defining the query with the appropriate select statement for the third question
query = "SELECT first_name, last_name \
         FROM song_play_song \
         WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Dropping the tables before closing out the sessions

In [20]:
## Dropping the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_play_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_play_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_play_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection

In [21]:
session.shutdown()
cluster.shutdown()